In [12]:
import omnipath
import pandas as pd
import liana

In [13]:
omnipath.__version__

'1.0.12'

# Pathways were created in OmnipathR, using the function kegg_pw_annot <- kegg_pathway_annotations() which creates a table of protein-pathway annotations 

In [14]:
pathways = pd.read_csv("../../data/LR_database/kegg_pathways.csv")

In [15]:
pathways

,Unnamed: 0,uniprot,genesymbol,pathway,pathway_id
0,1,Q03113,GNA12,MAPK signaling pathway,hsa04010
1,2,Q9Y4G8,RAPGEF2,MAPK signaling pathway,hsa04010
2,3,Q13972,RASGRF1,MAPK signaling pathway,hsa04010
3,4,O95267,RASGRP1,MAPK signaling pathway,hsa04010
4,5,P62834,RAP1A,MAPK signaling pathway,hsa04010
...,...,...,...,...,...
7529,7530,Q07812,BAX,Endocrine resistance,hsa01522
7530,7531,Q92934,BAD,Endocrine resistance,hsa01522
7531,7532,Q13323,BIK,Endocrine resistance,hsa01522
7532,7533,P46531,NOTCH1,Endocrine resistance,hsa01522


# Protein complexes

In [16]:
complexes = omnipath.requests.Complexes.get()

In [17]:
complexes

,name,components,components_genesymbols,stoichiometry,sources,references,identifiers,n_sources,n_primary_sources,n_references,references_stripped
0,NFY,P23511_P25208_Q13952,NFYA_NFYB_NFYC,1:1:1,CORUM;Compleat;ComplexPortal;PDB;SIGNOR;SPIKE;...,9372932;14755292;15243141,CORUM:4478;Compleat:HC1449;PDB:4awl;PDB:6qmp;P...,8,8,2,1;2
1,mTORC2,P42345_P68104_P85299_Q6R327_Q8TB45_Q9BVC4,DEPTOR_EEF1A1_MLST8_MTOR_PRR5_RICTOR,0:0:0:0:0:0,SIGNOR,None,SIGNOR:SIGNOR-C2,1,1,None,None
2,mTORC1,P42345_Q8N122_Q8TB45_Q96B36_Q9BVC4,AKT1S1_DEPTOR_MLST8_MTOR_RPTOR,0:0:0:0:0,SIGNOR,None,SIGNOR:SIGNOR-C3,1,1,None,None
3,SCF-betaTRCP,P63208_Q13616_Q9Y297,BTRC_CUL1_SKP1,1:1:1,CORUM;Compleat;SIGNOR;SPIKE,9990852,CORUM:227;Compleat:HC757;SIGNOR:SIGNOR-C5,4,4,1,2
4,CBP/p300,Q09472_Q92793,CREBBP_EP300,0:0,SIGNOR;hu.MAP3,None,SIGNOR:SIGNOR-C6,2,2,None,None
...,...,...,...,...,...,...,...,...,...,...,...
52080,ING6-MYST4,O95696_P55201_Q8WYB5_Q8WYH8,BRD1_BRPF1_ING5_KAT6B,0:0:0:0,SPIKE,None,None,1,1,None,None
52081,RNAPIIo,P19387_P24928_P30876,POLR2A_POLR2B_POLR2C,0:0:0,SPIKE,None,None,1,1,None,None
52082,ING5-MYST2,O95251_Q6IE81_Q8WYH8,ING5_JADE1_KAT7,0:0:0,SPIKE,None,None,1,1,None,None
52083,MCM,P25205_P33991_P33992_P33993_P49736_Q14566_Q7L590,MCM10_MCM2_MCM3_MCM4_MCM5_MCM6_MCM7,0:0:0:0:0:0:0,SPIKE,None,None,1,1,None,None


# all Omnipath Interactions

In [18]:
#import validated interactions 
resource = omnipath.interactions.OmniPath.get(entity_type="protein",genesymbol=True)
categories = omnipath.requests.Intercell.get()

In [19]:
resource = resource[["source_genesymbol","target_genesymbol","references"]]

In [20]:
resource.columns = ["from","to","references"]

In [105]:
resource.to_csv("../../data/LR_database/Omnipath_database.csv")

In [106]:
nodes = pd.DataFrame({"category":["Ligand"]*resource.shape[0] + ["Receptor"]*resource.shape[0], "identifier":resource["from"].tolist() + resource["to"].tolist()})

In [107]:
nodes = nodes.drop_duplicates("identifier")

In [108]:
nodes["Id"] = range(nodes.shape[0])

In [109]:
nodes = nodes[["Id","category","identifier"]]

In [110]:
nodes.to_csv("../../data/LR_database/OmniPath_nodes.csv")

In [111]:
nodes.index = nodes["identifier"].tolist()

In [112]:
interactions = resource.copy()

In [113]:
interactions["from"] = nodes.loc[interactions["from"].tolist()]["Id"].tolist()
interactions["to"] = nodes.loc[interactions["to"].tolist()]["Id"].tolist()

In [114]:
interactions.columns = ["Src","Dst","sources"]

In [116]:
interactions["edge_type"] = 1

In [118]:
interactions

,Src,Dst,sources,edge_type
0,0,10,TRIP:11290752;TRIP:11983166;TRIP:12601176,1
1,1,10,TRIP:11290752;TRIP:11983166;TRIP:12601176,1
2,2,10,TRIP:11290752;TRIP:11983166;TRIP:12601176,1
3,3,10,DIP:19897728;HPRD:12732636;IntAct:19897728;Lit...,1
4,4,10,TRIP:18261457,1
...,...,...,...,...
30048,25,2505,ProtMapper:26280531;iPTMnet:26280531,1
30049,850,387,ProtMapper:25605870;iPTMnet:25605870,1
30050,859,387,ProtMapper:27796369;iPTMnet:27796369,1
30051,2446,5995,iPTMnet:24070423,1


In [117]:
interactions.to_csv("../../data/LR_database/OmniPath_interactions.csv")

# Intercell interactions (for benchmarking)

In [3]:
#import validated interactions 
resource = omnipath.interactions.OmniPath.get(entity_type="protein",genesymbol=True)
categories = omnipath.requests.Intercell.get()

5.69MB [00:00, 10.9MB/s]


In [4]:
available_ligands = categories[categories["category"]=="ligand"]["genesymbol"].tolist()
available_receptors = categories[categories["category"]=="receptor"]["genesymbol"].tolist()

In [5]:
resource = resource[(resource["source_genesymbol"].isin(available_ligands)) & (resource["target_genesymbol"].isin(available_receptors))]

In [6]:
resource = resource[["source_genesymbol","target_genesymbol","references"]]

In [92]:
resource.to_csv("../../data/LR_database/intercell_Omnipath.csv")

In [8]:
resource

,source_genesymbol,target_genesymbol,references
26,EPOR,TRPC3,TRIP:18276585;TRIP:19074769;TRIP:21757714
38,FYN,TRPC4,TRIP:16144838;TRIP:18048348;dbPTM:16144838
48,FYN,TRPC6,HPRD:14761972;SPIKE_LC:16713569;TRIP:14761972
71,FYN,TRPV4,HPRD:12538589;SPIKE_LC:16713569;TRIP:12538589
88,KL,TRPV5,TRIP:16239475;TRIP:18495742;TRIP:18606998
...,...,...,...
29573,KDR,FLT4,HPRD:15474514;IntAct:16530705;IntAct:20224550;...
29776,MST1,STK38,ProtMapper:19062280;ProtMapper:28137909;iPTMne...
29791,FYN,STK11,ProtMapper:20142099;ProtMapper:21428801;ProtMa...
29872,TYK2,PRKD2,iPTMnet:21865166


In [13]:
intercell = intercell[["source_genesymbol","target_genesymbol","references"]]
intercell.columns = ["source","target","references"]

In [14]:
ppi = ppi[["from","to","references"]]
ppi.columns = ["source","target","references"]

In [19]:
resource = resource[(resource["source_genesymbol"].isin(categories["genesymbol"])) & (resource["target_genesymbol"].isin(categories["genesymbol"]))]

In [21]:
resource = resource[["source_genesymbol","target_genesymbol","references"]]
resource.columns = ["source","target","references"]

In [33]:
pathways = pd.read_csv("/h/soemily/new_GAT/data/LR_database/kegg_pathways.csv",index_col=0)

In [35]:
pathways.groupby("pathway").agg(list)

,uniprot,genesymbol,pathway_id
pathway,,,
AGE-RAGE signaling pathway in diabetic complications,"[P61812, P01137, P36897, Q15796, P30556, Q1510...","[TGFB2, TGFB1, TGFBR1, SMAD2, AGTR1, AGER, PRK...","[hsa04933, hsa04933, hsa04933, hsa04933, hsa04..."
AMPK signaling pathway,"[P41159, Q15848, Q15717, P05019, P08069, P0130...","[LEP, ADIPOQ, ELAVL1, IGF1, IGF1R, INS, INSR, ...","[hsa04152, hsa04152, hsa04152, hsa04152, hsa04..."
Acute myeloid leukemia,"[P42336, P10721, P36888, P42345, P62993, Q0788...","[PIK3CA, KIT, FLT3, MTOR, GRB2, SOS1, HGF, NRA...","[hsa05221, hsa05221, hsa05221, hsa05221, hsa05..."
Adherens junction,"[P08581, P00533, P08069, P16591, P06241, Q9H2G...","[MET, EGFR, IGF1R, FER, FYN, SLK, SRC, YES1, A...","[hsa04520, hsa04520, hsa04520, hsa04520, hsa04..."
Adipocytokine signaling pathway,"[P01375, P19438, P20333, Q04759, O15111, P1983...","[TNF, TNFRSF1A, TNFRSF1B, PRKCQ, CHUK, NFKB1, ...","[hsa04920, hsa04920, hsa04920, hsa04920, hsa04..."
...,...,...,...
Wnt signaling pathway,"[P49768, Q13485, O43318, Q9UBE8, Q9ULW2, P0463...","[PSEN1, SMAD4, MAP3K7, NLK, FZD10, TP53, DVL1,...","[hsa04310, hsa04310, hsa04310, hsa04310, hsa04..."
cAMP signaling pathway,"[P01215, P01303, Q9UBU3, P41586, P84996, O9546...","[CGA, NPY, GHRL, ADCYAP1R1, GNAS, GNAS, GNAS, ...","[hsa04024, hsa04024, hsa04024, hsa04024, hsa04..."
cGMP-PKG signaling pathway,"[P25100, P35348, P29992, Q13976, Q8TD86, Q0215...","[ADRA1D, ADRA1A, GNA11, PRKG1, CALML6, PRKCE, ...","[hsa04022, hsa04022, hsa04022, hsa04022, hsa04..."


In [36]:
pd.merge(ppi,intercell)

,source,target,references
0,EPOR,TRPC3,TRIP:18276585;TRIP:19074769;TRIP:21757714
1,FYN,TRPC4,TRIP:16144838;TRIP:18048348;dbPTM:16144838
2,FYN,TRPC6,HPRD:14761972;SPIKE_LC:16713569;TRIP:14761972
3,FYN,TRPV4,HPRD:12538589;SPIKE_LC:16713569;TRIP:12538589
4,KL,TRPV5,TRIP:16239475;TRIP:18495742;TRIP:18606998
...,...,...,...
3785,KDR,FLT4,HPRD:15474514;IntAct:16530705;IntAct:20224550;...
3786,MST1,STK38,ProtMapper:19062280;ProtMapper:28137909;iPTMne...
3787,FYN,STK11,ProtMapper:20142099;ProtMapper:21428801;ProtMa...
3788,TYK2,PRKD2,iPTMnet:21865166
